In [5]:
#!pip install arxiv --user

In [58]:
import pandas as pd
import arxiv

In [74]:
# i only use these if I want to remove annoying deprecation warnings from my analysis
import warnings
warnings.filterwarnings('ignore')

In [103]:
def search_arxiv(query, max_results=10):

    data = {}
    i = 0

    search = arxiv.Search(query=query, max_results=max_results)

    for result in search.results():
        
        try:

            data[i] = {}

            data[i]['title'] = result.title
            data[i]['date_published'] = result.published
            data[i]['authors'] = [a.name for a in result.authors]
            data[i]['summary'] = result.summary
            data[i]['url'] = result.pdf_url
            data[i]['category'] = result.primary_category
        
        except:
            
            print('weird arxiv error')
        
        # there are more fields that can be added; add as many as you need

        i += 1

    df = pd.DataFrame(data).T
    df = df[['date_published', 'title', 'authors', 'summary', 'url', 'category']]
    df['date_published'] = pd.to_datetime(df['date_published'])
    df.sort_values('date_published', ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [108]:
# at 1000 articles, it takes about 30 seconds
# at 10000 articles, it takes a while; put laptop down and walk away
# i haven't tried beyond 10000 yet

# at the moment, this is finicky beyond 10000; needs debugged or something
# could also be a problem with the arxiv library as it seems to be inside the search

# so far, i haven't noticed obvious throttling, but possibly throttling is causing the issue

query = 'Network Science'

max_results = 5000
    
df = search_arxiv(query, max_results) 
df.head()

,date_published,title,authors,summary,url,category
0,2023-11-01 05:04:55+00:00,Entity Alignment Method of Science and Technol...,"[Runze Fang, Yawen Li, Yingxia Shao, Zeli Guan...",The entity alignment of science and technology...,http://arxiv.org/pdf/2311.00300v1,cs.CL
1,2023-10-31 19:51:24+00:00,"Statistical Network Analysis: Past, Present, a...",[Srijan Sengupta],This article provides a brief overview of stat...,http://arxiv.org/pdf/2311.00122v1,stat.ME
2,2023-10-31 09:33:03+00:00,A physics-informed GAN Framework based on Mode...,"[Kerem Ciftci, Klaus Hackl]",Model-free data-driven computational mechanics...,http://arxiv.org/pdf/2310.20308v1,cs.CE
3,2023-10-31 09:11:12+00:00,Verification of Neural Networks Local Differen...,"[Roie Reshef, Anan Kabaha, Olga Seleznova, Dan...",Neural networks are susceptible to privacy att...,http://arxiv.org/pdf/2310.20299v1,cs.LG
4,2023-10-30 17:58:01+00:00,iGEM: a model system for team science and inno...,"[Marc Santolini, Leo Blondel, Megan J. Palmer,...",Teams are a primary source of innovation in sc...,http://arxiv.org/pdf/2310.19858v1,cs.SI


In [111]:
df.shape

(5000, 6)

In [112]:
outfile = 'data/arxiv_network_science_data.csv'

df.to_csv(outfile, index=False)

In [ ]:
# occasionally randomly crashes; need to find a fix; maybe others can help